In [ ]:
#!pip install cobra
#!pip install matplotlib
!pip install matplotlib-venn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for matplotlib-venn: filename=matplotlib_venn-1.1.2-py3-none-any.whl size=45388 sha256=d935b3a9a3bcac8f4bbfd1d63c617bb852fe3d2e3558968d9f06961603038ed6
  Stored in directory: /home/codespace/.cache/pip/wheels/c2/47/0c/f014c55a1cfd56dce41a1cafd23e3c590652b5e71330cc181c
Successfully built matplotlib-venn

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import pandas as pd 
import math
import sys

import numpy as np
from scipy.stats import ttest_ind
import cobra
from matplotlib_venn import venn3
import matplotlib.pyplot as plt

source_dir = 'src'
sys.path.append(source_dir)

from transcriptomics.get_gene_transcript_dictionary import get_gene_transcript_dictionary
from transcriptomics.get_reaction_transcript_dictionary import get_reaction_transcript_dictionary

In [3]:
fpkm_df = pd.read_csv("fpkm_counts_with_annotations.csv")

model = cobra.io.json.load_json_model('models/iYLI647_corr_3.json')

In [4]:
fpkm_df.head()

,Geneid,EBY_1,EBY_2,EBY_3,EBY-U_1,EBY-GNSR_1,EBY-GNSR_2,EBY-GNHR_1,EBY-GNHR_2,EBY-GNHR_3,EBY-GNUR_1,EBY-GNUR_2,EBY-GNUR_3,JGI_annotation,KEGG_annotation
0,jgi|Sacce1|1000|YDL018C,47.55,49.28,46.03,27.44,66.92,56.75,26.36,30.20,32.08,45.72,44.23,46.24,NaN,no KO assigned | (RefSeq) ERP3; Erp3p
1,jgi|Sacce1|1001|YDL017W,20.11,20.75,19.72,26.48,25.70,28.76,22.21,20.59,22.80,33.20,36.83,33.11,Protein kinase,K02214 cell division control protein 7 [EC:2.7...
2,jgi|Sacce1|1002|YDL016C,5.68,6.50,3.91,4.32,7.41,7.81,2.39,2.12,2.39,5.77,2.15,3.29,NaN,NaN
3,jgi|Sacce1|1003|YDL015C,258.04,263.32,271.20,176.40,287.94,309.80,97.96,133.22,114.73,172.79,189.77,190.81,3-oxo-5-alpha-steroid 4-dehydrogenase,K10258 very-long-chain enoyl-CoA reductase [EC...
4,jgi|Sacce1|1004|YDL014W,338.89,352.15,227.14,32.43,264.66,275.53,33.94,28.27,20.38,70.73,106.82,72.10,NaN,K14563 rRNA 2'-O-methyltransferase fibrillarin...


In [7]:


fpkm_df.head()

,Geneid,EBY_1,EBY_2,EBY_3,EBY-U_1,EBY-GNSR_1,EBY-GNSR_2,EBY-GNHR_1,EBY-GNHR_2,EBY-GNHR_3,EBY-GNUR_1,EBY-GNUR_2,EBY-GNUR_3,JGI_annotation,KEGG_annotation
0,jgi|Sacce1|1000|YDL018C,47.55,49.28,46.03,27.44,66.92,56.75,26.36,30.20,32.08,45.72,44.23,46.24,NaN,no KO assigned | (RefSeq) ERP3; Erp3p
1,jgi|Sacce1|1001|YDL017W,20.11,20.75,19.72,26.48,25.70,28.76,22.21,20.59,22.80,33.20,36.83,33.11,Protein kinase,K02214 cell division control protein 7 [EC:2.7...
2,jgi|Sacce1|1002|YDL016C,5.68,6.50,3.91,4.32,7.41,7.81,2.39,2.12,2.39,5.77,2.15,3.29,NaN,NaN
3,jgi|Sacce1|1003|YDL015C,258.04,263.32,271.20,176.40,287.94,309.80,97.96,133.22,114.73,172.79,189.77,190.81,3-oxo-5-alpha-steroid 4-dehydrogenase,K10258 very-long-chain enoyl-CoA reductase [EC...
4,jgi|Sacce1|1004|YDL014W,338.89,352.15,227.14,32.43,264.66,275.53,33.94,28.27,20.38,70.73,106.82,72.10,NaN,K14563 rRNA 2'-O-methyltransferase fibrillarin...


In [8]:
EBY_1_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY_1')
EBY_2_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY_2')
EBY_3_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY_3')

EBY_U_1_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-U_1')

EBY_GNSR_1_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNSR_1')
EBY_GNSR_2_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNSR_2')

EBY_GNHR_1_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNHR_1')
EBY_GNHR_2_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNHR_2')
EBY_GNHR_3_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNHR_3')

EBY_GNUR_1_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNUR_1')
EBY_GNUR_2_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNUR_2')
EBY_GNUR_3_gene_transcripts = get_gene_transcript_dictionary(fpkm_df, 'EBY-GNUR_3')


EBY_1_gene_transcripts

KeyError: 'YALI0_id'